In [3]:
import pandas as pd
from tqdm.auto import tqdm
import actions
from pathlib import Path
from utils import *
import re
from tqdm.auto import tqdm

In [4]:
# Define the file paths
parquet_file_abcd = Path("../data/ABCD_tripfiles.parquet")
parquet_file_abcd_conv = Path("../data/ABCD_tripfiles_conv.parquet")
parquet_file_mnop = Path("../data/MNOP_tripfiles.parquet")
parquet_file_mnop_conv = Path("../data/MNOP_tripfiles_conv.parquet")
parquet_file_zyxw = Path("../data/ZYXW_tripfiles.parquet")
parquet_file_zyxw_conv = Path("../data/ZYXW_tripfiles_conv.parquet")
print(
    parquet_file_abcd,
    parquet_file_abcd_conv,
    parquet_file_mnop,
    parquet_file_mnop_conv,
    parquet_file_zyxw,
    parquet_file_zyxw_conv,
)

..\data\ABCD_tripfiles.parquet ..\data\ABCD_tripfiles_conv.parquet ..\data\MNOP_tripfiles.parquet ..\data\MNOP_tripfiles_conv.parquet ..\data\ZYXW_tripfiles.parquet ..\data\ZYXW_tripfiles_conv.parquet


In [5]:
df_abcd = pd.read_parquet(parquet_file_abcd)
df_mnop = pd.read_parquet(parquet_file_mnop)
df_zyxw = pd.read_parquet(parquet_file_zyxw)

In [6]:
print(round(df_abcd.memory_usage(deep=True).sum() / 1024**2, 2), "MB")
print(round(df_mnop.memory_usage(deep=True).sum() / 1024**2, 2), "MB")
print(round(df_zyxw.memory_usage(deep=True).sum() / 1024**2, 2), "MB")

1252.74 MB
2278.75 MB
608.15 MB


In [7]:
action_names = set(df_abcd.action_name.unique())
len(action_names)

49

In [8]:
action_names.update(set(df_mnop.action_name.unique()))
action_names.update(set(df_zyxw.action_name.unique()))

In [9]:
action_extractors = {
    "CalculateWeightAndTrimAction": None,
    "CheckinMsgProcessor": None,
    "CreateLoadsheetAction": None,
    "CreateZFWMessageAction": None,
    "SendFuelOrderAction": None,
    "SendLoadsheetAction": None,
    "SetActualBagWeightIndicatorAction": None,
    "StorePaxDataAction": None,
    "StoreRegistrationAndConfigurationAc": None,
    "UpdateFuelDataAction": None,
    "RampFinalAction": actions.RampFinalAction.extract,
    "CreateLoadingInstructionAction": None,
    "CopyPaxDataAction": None,
    "CreateBaggageLoadItemsAction": None,
    "EstimateStorePaxDataAction": None,
    "SendLoadingInstructionAction": None,
    "SetCKIPaxDistributionAction": actions.SetCKIPaxDistributionAction.extract,
    "SpecialPaxWeightAction": None,
    "StoreAircraftDataAction": None,
    "StoreCKIAverageWeightAction": None,
    "StorePaxDataGuiAction": None,
    "TdmCreateLoadingInstructionAction": None,
    "TransferCargoAction": None,
    "TransferCheckinDataAction": None,
    "UpdateCargoMailEstimatesAction": None,
    "UpdateEstimatesAction": None,
    "UpdateLastEzfwSentAction": None,
    "UpdateLoadTableAction": None,
}

In [10]:
def extract_df(df: pd.DataFrame):
    tqdm.pandas()
    for action_name, extractor in action_extractors.items():

        if extractor is not None:
            print(action_name)

            df[f"data_{action_name}"] = df[df.action_name == action_name][
                "entry_details"
            ].progress_apply(extractor)
    return df


df_abcd_conv = extract_df(df_abcd)
df_abcd_conv.to_parquet(parquet_file_abcd_conv, engine="pyarrow", compression="brotli")

df_mnop_conv = extract_df(df_mnop)
df_mnop_conv.to_parquet(parquet_file_mnop_conv, engine="pyarrow", compression="brotli")

df_zyxw_conv = extract_df(df_zyxw)
df_zyxw_conv.to_parquet(parquet_file_zyxw_conv, engine="pyarrow", compression="brotli")

RampFinalAction


100%|██████████| 3804/3804 [00:00<00:00, 63165.38it/s]


SetCKIPaxDistributionAction


0it [00:00, ?it/s]


RampFinalAction


100%|██████████| 5612/5612 [00:00<00:00, 62321.44it/s]


SetCKIPaxDistributionAction


0it [00:00, ?it/s]


KeyboardInterrupt: 

In [ ]:
df = df_zyxw


x = df[
    (df.action_name == "SetCKIPaxDistributionAction")
    # & (~df["entry_details"].isnull())
    # & (~df["entry_details"].str.contains("Discrepancy check result", na=False))
    # & (~df["entry_details"].str.contains("Caller user", na=False))  # don't include this
]["entry_details"].unique()

In [ ]:
"""TOTAL Pax: 6    Y: 6  Jump: 0    StandBy: NULL  Male: 2  Female: 2  Child: 2  Infant: 0  Total bag: 6  Total bag weight: 72.0 KG  Baggage weight type: HISTORIC\r\n 

Distribution        : CKI_DISTRIBUTION         \r\n 
Section             : 0A                       0C                       \r\n 
Capacity            : Y72                      Y102                     \r\n 
Distribution        : Y0                       Y0"""

In [ ]:
foo: set = set()

In [ ]:
for message in x:
    items = message.split(" ")
    for item in items:
        if not item.isdigit():
            foo.add(item)

In [ ]:
foo

{'AIRBORNE',
 'AIRCRAFT_CONFIG',
 'ALLOWANCE_CHECK_PERFORMED',
 'AUTOMATION_STARTED',
 'AUTO_MODE_ACTIVE',
 'BAG_LOAD_ITEMS_GEN',
 'BAG_LOZY_ITEMS_GEN',
 'BAG_ULD_ORD',
 'CABIN_CONFIG',
 'CALC_HIST_DATA',
 'CARGO_FINAL',
 'CARGO_TRANSFER',
 'CHECK_IN_FINAL',
 'CHECK_IN_OPEN',
 'DGR_ITEMS',
 'EZFW',
 'EZFW_COUNTER',
 'FINAL_RELEASE',
 'FUEL',
 'FUEL_ORDER',
 'LDM',
 'LOADING_INSTRUCTION',
 'LOADSHEET',
 'LOZYING_INSTRUCTION',
 'LOZYSHEET',
 'OFFBLOCK',
 'OFP',
 'PDM',
 'RAMP_FINAL',
 'REGISTRATION',
 'REGISTRATION_CHANGE',
 'STATUS',
 'TRANSIT_ACCEPTANCE',
 'TRANSIT_PAX'}

In [ ]:
extract(df_zyxw[df_zyxw.id == 375438310]["entry_details"].iloc(0)[0])

{'TOTAL Pax': None,
 'Y': 0,
 'Jump': None,
 'StandBy': None,
 'Male': None,
 'Female': None,
 'Child': None,
 'Infant': None,
 'Total bag': 0,
 'Total bag weight': 0.0,
 'Baggage weight type': 'HISTORIC',
 'Weight unit': 'KG',
 'Sections': {'0A': {'Capacity': 'Y72', 'Distribution': 'Y0'},
  '0C': {'Capacity': 'Y102', 'Distribution': 'Y0'}},
 'Distribution': 'CKI_DISTRIBUTION'}

In [ ]:
df_zyxw[df_zyxw.id == 375438310]["entry_details"].iloc(0)[0]

'TOTAL Pax: NULL    Y: 0  Jump: NULL    StandBy: NULL  Male: NULL  Female: NULL  Child: NULL  Infant: NULL  Total bag: 0  Total bag weight: 0.0 KG  Baggage weight type: HISTORIC\r\n Distribution        : CKI_DISTRIBUTION         \r\n Section             : 0A                       0C                       \r\n Capacity            : Y72                      Y102                     \r\n Distribution        : Y0                       Y0'

In [ ]:
x[x.action_name == "SetCKIPaxDistributionAction"].iloc(0)[8836]

id                                                           375438310
creation_time                                      2024-05-05 05:06:37
airline_code                                                        ZY
flight_number                                                     5032
flight_date                                                          5
departure_airport                                                  REC
user_name                                                 service-acco
action_name                                SetCKIPaxDistributionAction
header_line          2024-05-05 05:06:37,437 INFO  [b871eebd4cb0966...
entry_details        TOTAL Pax: NULL    Y: 0  Jump: NULL    StandBy...
Name: 334375, dtype: object

In [ ]:
x = df_zyxw.copy()
x["data_SetCKIPaxDistributionAction"] = x[
    x.action_name == "SetCKIPaxDistributionAction"
]["entry_details"].progress_apply(extract)

100%|██████████| 11598/11598 [00:00<00:00, 53015.90it/s]
